In [1]:
import spotipy
import pandas as pd
import plotly.express as px
import seaborn as sns
import sys
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
client_credentials_manager = SpotifyClientCredentials(client_id='c7832f4cfebb4553b27447fe32b29c1d', client_secret='d61c1c95e44446bdab707511f8d306ae')
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
playlist_link = "https://open.spotify.com/playlist/2QECvpSDU035PFS5uJ8x3l?si=5730077d7aca4db3"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
# Function to extract MetaData from a playlist thats longer than 100 songs
def get_playlist_tracks_more_than_100_songs(username, playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        print(i) # Counter
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
                           'title':[playlist_tracks_titles],
                           'all_artists':[playlist_tracks_artists],
                           'popularity':[playlist_tracks_popularity],
                           'release_date':[playlist_tracks_first_release_date],
                           'danceability':[features[0]['danceability']],
                           'energy':[features[0]['energy']],
                           'key':[features[0]['key']],
                           'loudness':[features[0]['loudness']],
                           'mode':[features[0]['mode']],
                           'acousticness':[features[0]['acousticness']],
                           'instrumentalness':[features[0]['instrumentalness']],
                           'liveness':[features[0]['liveness']],
                           'valence':[features[0]['valence']],
                           'tempo':[features[0]['tempo']],
                           'duration_ms':[features[0]['duration_ms']],
                           'time_signature':[features[0]['time_signature']]
                           }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue

    return features_df

In [ ]:
get_playlist_tracks_more_than_100_songs('c7832f4cfebb4553b27447fe32b29c1d','2QECvpSDU035PFS5uJ8x3l')

In [ ]:
#Terminal Command for downloading songs from Spotify
#prerequisites: ffmpeg library and spotify_dl library
pip3 install spotify_dl
spotify_dl -l spotify_playlist_link/s -o download_directory


In [ ]:
#Script for renaming trimming the audio files into clips of 30 seconds
#Trimming the songs to small clips of 30 seconds, for making the audio processing easier 

from pydub import AudioSegment

path_of_the_directory ='/home/asmaosman/Music/Datasets/Sudanese_Dataset/'
ext = ('.mp3')
i=0
for files in os.listdir(path_of_the_directory):
    if files.endswith(ext):

        startMin = 0
        startSec = 180

        endMin = 0
        endSec = 210

        # Time to miliseconds
        startTime = startMin*60*1000+startSec*1000
        endTime = endMin*60*1000+endSec*1000

        # Opening file and extracting segment
        song = AudioSegment.from_mp3(path_of_the_directory+files)
        extract = song[startTime:endTime]

        # Saving
        
        extract.export( "/Sudanese_Dataset/middle_30_seconds/middle"+str(i)+".wav", format="wav")
        i=i+1
    else:
        continue

In [ ]:
#Renaming files in a chronological order
import os


path = '/Music/Sudanese/'

i = 0
for filename in os.listdir(path):
  os.rename(path+'/'+filename, path+'/'+str(i)+'.mp3')
  i = i +1